# BERT Chunks Classification pipeline
X - Code Chunks

y - Chunk Labels (generated by regex)

## Modules

In [ ]:
!pip install tensorflow_hub==0.8
!pip install tensorflow==2.1.0
!pip install tensorflow-gpu==2.1.0
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 421.8MB 40kB/s 
     |████████████████████████████████| 3.9MB 3.7MB/s 
     |████████████████████████████████| 450kB 23.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=4be18117f55cd6e452e88d0b762289da922b7b6bbe040a2532a1041a3e3cc970
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: t

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import keras
print("TF version: ", tf.__version__)
print("Hub version: ", hub.__version__)
import bert
from tensorflow.keras.models import Model       # Keras is the new high level API for TensorFlow
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.stats import wasserstein_distance

TF version:  2.1.0
Hub version:  0.8.0


Using TensorFlow backend.


In [ ]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

## Functions

In [ ]:
def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids
def tokenize_sentence(sentence):
    stokens = tokenizer.tokenize(sentence)
    stokens = ["[CLS]"] + stokens + ["[SEP]"]
    
    input_ids = get_ids(stokens, tokenizer, max_seq_length)
    input_masks = get_masks(stokens, max_seq_length)
    input_segments = get_segments(stokens, max_seq_length)
    
    return input_ids, input_masks, input_segments

def compare_sentences(sentence_1, sentence_2, distance_metric):
    input_ids_1, input_masks_1, input_segments_1 = tokenize_sentence(sentence_1)
    input_ids_2, input_masks_2, input_segments_2 = tokenize_sentence(sentence_2)
    
    pool_embs_1, all_embs_1 = model.predict([[input_ids_1],[input_masks_1],[input_segments_1]])
    pool_embs_2, all_embs_2 = model.predict([[input_ids_2],[input_masks_2],[input_segments_2]])
#     print(pool_embs_1, all_embs_1)
#     print(pool_embs_2, all_embs_2)
    return distance_metric(pool_embs_1[0], pool_embs_2[0])

def distance_between_sentences(sentence_1, sentence_2, distance_metric):
    input_ids_1, input_masks_1, input_segments_1 = tokenize_sentence(sentence_1)
    input_ids_2, input_masks_2, input_segments_2 = tokenize_sentence(sentence_2)
    pool_embs_1, all_embs_1 = model.predict([[input_ids_1],[input_masks_1],[input_segments_1]])
    pool_embs_2, all_embs_2 = model.predict([[input_ids_2],[input_masks_2],[input_segments_2]])
    distances = []
    for i in range(0,max_seq_length):
      distances.append(distance_metric(all_embs_1[0][i], all_embs_2[0][i]))
    distance = np.mean(distances)
    return distance

def get_embs(sentence):
    input_ids, input_masks, input_segments = tokenize_sentence(sentence)
    pool_embs, all_embs = model.predict([[input_ids],[input_masks],[input_segments]])
    return pool_embs, all_embs

def get_all_embs(sentence):
    input_ids, input_masks, input_segments = tokenize_sentence(sentence)
    pool_embs, _ = model.predict([[input_ids],[input_masks],[input_segments]])
    return pool_embs

def square_rooted(x):
    return math.sqrt(sum([a*a for a in x]))

def cosine_similarity(x,y):
    numerator = sum(a*b for a,b in zip(x,y))
    denominator = square_rooted(x)*square_rooted(y)
    return 1-numerator/float(denominator)

def dummy_metric(x,y):
    return 42

def create_single_input(sentence, MAX_LEN):
  
    stokens = tokenizer.tokenize(sentence)
    
    stokens = stokens[:MAX_LEN]
    
    stokens = ["[CLS]"] + stokens + ["[SEP]"]
  
    ids = get_ids(stokens, tokenizer, max_seq_length)
    masks = get_masks(stokens, max_seq_length)
    segments = get_segments(stokens, max_seq_length)

    return ids,masks,segments

def create_input_array(sentences):

    input_ids, input_masks, input_segments = [], [], []

    for sentence in tqdm(sentences, position=0, leave=True):
        ids,masks,segments=create_single_input(sentence, max_seq_length-2)

        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)

    return [np.asarray(input_ids, dtype=np.int32), 
              np.asarray(input_masks, dtype=np.int32), 
              np.asarray(input_segments, dtype=np.int32)]

## Data Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
KK_PATH = '/content/'
# DATASET_PATH = 'code_blocks_final_clean.csv' ## CODE_COLUMN = 'code_block'
DATASET_PATH = 'chunks_30_final.csv' ## CODE_COLUMN = 'code'
# DATASET_PATH = 'nl2ml preprocessing (united).csv' ## CODE_COLUMN = 'Code'
NEW_DATASET_PATH = DATASET_PATH[:-4] + '_regex.csv'
CODE_COLUMN = 'code'

In [ ]:
# df = pd.read_csv(DATASET_PATH)
df = pd.read_csv(NEW_DATASET_PATH)

In [ ]:
# df.drop('label', axis=1, inplace=True)

In [ ]:
df['label'] = df['preprocessing'].astype('str')# + df['tag_visualization'].astype('str')

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['label'] = le.fit_transform(df['label'])

In [ ]:
df['preprocessing'].value_counts()

1    49112
0    27131
Name: preprocessing, dtype: int64

In [ ]:
# df['tag_import_output'].value_counts()

In [ ]:
X = df['code'].values
class_names = df['label'].unique()
class_ids = [class_id for class_id,class_name in enumerate(class_names)]
y = df['label'].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

## Compiling BERT


In [ ]:
%time
max_seq_length = 512

input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="segment_ids")

BERT_HUB_URL = 'https://tfhub.dev/tensorflow/bert_en_cased_L-24_H-1024_A-16/2'
bert_layer = hub.KerasLayer(BERT_HUB_URL, trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=[pooled_output, sequence_output])

x = tf.keras.layers.GlobalAveragePooling1D()(sequence_output)
x = tf.keras.layers.Dropout(0.2)(x)
out = tf.keras.layers.Dense(len(class_names), activation="sigmoid", name="dense_output")(x)
pred = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(
      inputs=[input_word_ids, input_mask, segment_ids],
      outputs=out)

model.compile(loss='sparse_categorical_crossentropy', #'binary_crossentropy'
                  optimizer='adam',
                  metrics=['accuracy'])

model.summary()

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
FullTokenizer = bert.bert_tokenization.FullTokenizer
tokenizer = FullTokenizer(vocab_file, do_lower_case)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.44 µs
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 512)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      [(None, 1024), (None 333579265   input_word_ids[0][0]             
                            

In [ ]:
train_inputs = create_input_array(X_train)

100%|██████████| 68618/68618 [05:01<00:00, 227.85it/s]


## BERT Training (fine-tuning)

In [ ]:
MODEL_PATH = '/content/drive/My Drive/NL2ML/BERT_weights/'
WEIGHTS_PATH = MODEL_PATH + 'NL2ML_BERT_9.h5'

In [ ]:
class CustomModelCheckPoint(keras.callbacks.Callback):
    def __init__(self,**kargs):
        super(CustomModelCheckPoint,self).__init__(**kargs)
        self.epoch_accuracy = {} # loss at given epoch
        self.epoch_loss = {} # accuracy at given epoch

    def on_epoch_begin(self,epoch, logs={}):
        # Things done on beginning of epoch. 
        return

    def on_epoch_end(self, epoch, logs={}):
        # things done on end of the epoch
        # self.epoch_accuracy[epoch] = logs.get("acc")
        # self.epoch_loss[epoch] = logs.get("loss")
        self.model.save_weights(MODEL_PATH+"NL2ML_BERT_%d.h5" %epoch) # save the model

In [ ]:
checkpoint = CustomModelCheckPoint()

In [ ]:
# from keras.models import load_model
# model = load_model(WEIGHTS_PATH)

In [ ]:
history = model.fit(train_inputs, y_train, epochs=10, batch_size=30,
                    callbacks=[checkpoint], validation_split=0.1, verbose=1,
                    steps_per_epoch=None)

In [ ]:
best_epoch = np.argmin(history.history['val_loss']) + 1
best_epoch

## BERT Predicting

In [ ]:
# model.history.history['val_loss']

In [ ]:
test_inputs = create_input_array(X_test)

100%|██████████| 7625/7625 [00:34<00:00, 218.64it/s]


In [ ]:
predictions = model.predict(test_inputs)

## Layout

In [ ]:
# print(predictions.shape)
# for i in range(0,20):
#     print(class_names[np.argmax(predictions[i])])

In [ ]:
y_pred = []
for i in range(0,len(predictions)):
    y_pred.append(class_names[np.argmax(predictions[i])])
y_pred = np.array(y_pred)

In [ ]:
np.unique(y_pred)

array([0, 1])

In [ ]:
from sklearn import metrics
print(100*round(metrics.f1_score(y_test, y_pred, average='weighted'),2),"%")
print(100*round(metrics.accuracy_score(y_test, y_pred),2),"%")

55.00000000000001 %
61.0 %
